In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from data_loader import DataLoader
import numpy as np
import os



## sigin
import signal
import sys

Using TensorFlow backend.


In [2]:
## source from keras-gan

class colorizeSketch():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)
        

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50,load=False):

        start_time = datetime.datetime.now()


        if load == True:
            1+1
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        
        Gloss=100
        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

                
                # If at save interval => save generated image samples
                if batch_i % 4 == 0 and epoch % 10 == 0  :
                    self.sample_images(epoch, batch_i)
                if g_loss[0] < Gloss :
                    self.g_AB.save('./models/gen_model.h5')
                    self.g_AB.save_weights('./models/gen_weights.h5')
                    self.combined.save('./models/model.h5')
                    self.combined.save_weights('./models/weights.h5')
                    Gloss = g_loss[0] 

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

def signal_handler(self,sig, frame):
        print('Graceful Interupt')


In [ ]:

if __name__ == '__main__':


    gan = colorizeSketch()
    
    gan.train(epochs=100, batch_size=3, sample_interval=100,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/278] time: 0:01:33.836795, [d_loss: 28.354725, g_loss: 112.704742]
[0] [1/278] time: 0:01:35.724061, [d_loss: 32.449013, g_loss: 136.212524]
[0] [2/278] time: 0:01:36.959289, [d_loss: 35.028946, g_loss: 15.577663]
[0] [3/278] time: 0:01:53.948734, [d_loss: 6.908571, g_loss: 4.905550]
[0] [4/278] time: 0:02:00.398747, [d_loss: 1.528908, g_loss: 4.761908]
[0] [5/278] time: 0:02:07.570089, [d_loss: 0.595515, g_loss: 4.724053]
[0] [6/278] time: 0:02:14.203060, [d_loss: 0.619705, g_loss: 5.112312]
[0] [7/278] time: 0:02:15.443563, [d_loss: 0.455823, g_loss: 5.712058]
[0] [8/278] time: 0:02:16.687861, [d_loss: 0.568342, g_loss: 4.567587]
[0] [9/278] time: 0:02:24.118075, [d_loss: 0.769455, g_loss: 5.510798]
[0] [10/278] time: 0:02:25.389131, [d_loss: 0.333366, g_loss: 4.286175]
[0] [11/278] time: 0:02:34.324605, [d_loss: 0.215233, g_loss: 4.383053]
[0] [12/278] time: 0:02:35.538776, [d_loss: 0.204584, g_loss: 4.598567]
[0] [13/278] time: 0:02:37.123456, [d_loss: 0.414084, g_loss: 3.65

[0] [114/278] time: 0:05:37.449352, [d_loss: 0.304545, g_loss: 2.930466]
[0] [115/278] time: 0:05:38.664726, [d_loss: 0.274727, g_loss: 2.841469]
[0] [116/278] time: 0:05:39.859530, [d_loss: 0.356559, g_loss: 2.760008]
[0] [117/278] time: 0:05:41.409580, [d_loss: 0.400938, g_loss: 2.454463]
[0] [118/278] time: 0:05:42.590140, [d_loss: 0.362055, g_loss: 2.657859]
[0] [119/278] time: 0:05:43.768194, [d_loss: 0.417021, g_loss: 2.596565]
[0] [120/278] time: 0:05:44.944886, [d_loss: 0.445773, g_loss: 2.272656]
[0] [121/278] time: 0:05:52.222867, [d_loss: 0.275516, g_loss: 2.634515]
[0] [122/278] time: 0:05:53.410004, [d_loss: 0.433833, g_loss: 1.870053]
[0] [123/278] time: 0:06:00.321714, [d_loss: 0.410387, g_loss: 2.347584]
[0] [124/278] time: 0:06:01.500509, [d_loss: 0.334453, g_loss: 2.604024]
[0] [125/278] time: 0:06:03.027350, [d_loss: 0.394861, g_loss: 2.458895]
[0] [126/278] time: 0:06:04.218114, [d_loss: 0.280226, g_loss: 2.788971]
[0] [127/278] time: 0:06:05.404941, [d_loss: 0.3417

[0] [227/278] time: 0:08:24.753530, [d_loss: 0.364168, g_loss: 2.156168]
[0] [228/278] time: 0:08:25.940679, [d_loss: 0.366646, g_loss: 2.139067]
[0] [229/278] time: 0:08:27.506173, [d_loss: 0.300132, g_loss: 2.148498]
[0] [230/278] time: 0:08:28.697938, [d_loss: 0.405020, g_loss: 1.914601]
[0] [231/278] time: 0:08:29.884872, [d_loss: 0.427204, g_loss: 2.428478]
[0] [232/278] time: 0:08:31.081628, [d_loss: 0.412047, g_loss: 2.029418]
[0] [233/278] time: 0:08:32.655611, [d_loss: 0.395739, g_loss: 1.890754]
[0] [234/278] time: 0:08:33.855844, [d_loss: 0.295208, g_loss: 2.153497]
[0] [235/278] time: 0:08:35.057948, [d_loss: 0.357456, g_loss: 1.761273]
[0] [236/278] time: 0:08:36.255035, [d_loss: 0.327927, g_loss: 1.572009]
[0] [237/278] time: 0:08:37.827123, [d_loss: 0.309298, g_loss: 2.033520]
[0] [238/278] time: 0:08:39.019868, [d_loss: 0.351423, g_loss: 1.882016]
[0] [239/278] time: 0:08:40.208267, [d_loss: 0.293620, g_loss: 2.422067]
[0] [240/278] time: 0:08:41.389764, [d_loss: 0.5398

[1] [64/278] time: 0:10:45.551186, [d_loss: 0.275811, g_loss: 1.878384]
[1] [65/278] time: 0:10:46.745234, [d_loss: 0.331749, g_loss: 1.718245]
[1] [66/278] time: 0:10:47.928421, [d_loss: 0.288403, g_loss: 2.012295]
[1] [67/278] time: 0:10:49.115288, [d_loss: 0.319925, g_loss: 1.916503]
[1] [68/278] time: 0:10:50.300656, [d_loss: 0.285312, g_loss: 2.065731]
[1] [69/278] time: 0:10:51.481742, [d_loss: 0.500041, g_loss: 1.661636]
[1] [70/278] time: 0:10:52.665417, [d_loss: 0.241291, g_loss: 2.214079]
[1] [71/278] time: 0:10:53.841977, [d_loss: 0.353513, g_loss: 1.744513]
[1] [72/278] time: 0:10:55.028047, [d_loss: 0.277953, g_loss: 1.810024]
[1] [73/278] time: 0:10:56.217139, [d_loss: 0.306336, g_loss: 2.156269]
[1] [74/278] time: 0:10:57.399378, [d_loss: 0.372581, g_loss: 1.881238]
[1] [75/278] time: 0:10:58.578952, [d_loss: 0.446089, g_loss: 2.126729]
[1] [76/278] time: 0:10:59.759605, [d_loss: 0.445019, g_loss: 1.677724]
[1] [77/278] time: 0:11:00.942823, [d_loss: 0.385388, g_loss: 1.

...

[5] [203/278] time: 0:36:21.723837, [d_loss: 0.288389, g_loss: 1.151113]
[5] [204/278] time: 0:36:22.902461, [d_loss: 0.292233, g_loss: 1.073726]
[5] [205/278] time: 0:36:24.097274, [d_loss: 0.293814, g_loss: 1.611930]
[5] [206/278] time: 0:36:25.277272, [d_loss: 0.292713, g_loss: 1.317810]
[5] [207/278] time: 0:36:26.455740, [d_loss: 0.299140, g_loss: 1.580971]
[5] [208/278] time: 0:36:27.638020, [d_loss: 0.280105, g_loss: 1.341419]
[5] [209/278] time: 0:36:28.816591, [d_loss: 0.285609, g_loss: 1.199817]
[5] [210/278] time: 0:36:30.005613, [d_loss: 0.274824, g_loss: 1.259793]
[5] [211/278] time: 0:36:31.193683, [d_loss: 0.289322, g_loss: 1.002740]
[5] [212/278] time: 0:36:32.384043, [d_loss: 0.287762, g_loss: 1.085352]
[5] [213/278] time: 0:36:33.570576, [d_loss: 0.287308, g_loss: 1.095770]
[5] [214/278] time: 0:36:34.757150, [d_loss: 0.286574, g_loss: 0.967339]
[5] [215/278] time: 0:36:40.827954, [d_loss: 0.259383, g_loss: 1.509109]
[5] [216/278] time: 0:36:42.009786, [d_loss: 0.2876

[6] [39/278] time: 0:38:40.366459, [d_loss: 0.280651, g_loss: 1.171426]
[6] [40/278] time: 0:38:41.548259, [d_loss: 0.284668, g_loss: 1.118760]
[6] [41/278] time: 0:38:42.726050, [d_loss: 0.266774, g_loss: 1.167896]
[6] [42/278] time: 0:38:43.908554, [d_loss: 0.280158, g_loss: 1.175013]
[6] [43/278] time: 0:38:45.102321, [d_loss: 0.268593, g_loss: 1.239715]
[6] [44/278] time: 0:38:46.288640, [d_loss: 0.291036, g_loss: 1.370636]
[6] [45/278] time: 0:38:47.474241, [d_loss: 0.340522, g_loss: 1.298854]
[6] [46/278] time: 0:38:48.652773, [d_loss: 0.306313, g_loss: 1.395236]
[6] [47/278] time: 0:38:49.834294, [d_loss: 0.293070, g_loss: 1.196605]
[6] [48/278] time: 0:38:51.023243, [d_loss: 0.286381, g_loss: 1.384901]
[6] [49/278] time: 0:38:52.212187, [d_loss: 0.296112, g_loss: 1.156332]
[6] [50/278] time: 0:38:53.395514, [d_loss: 0.282822, g_loss: 1.262397]
[6] [51/278] time: 0:38:54.578983, [d_loss: 0.283521, g_loss: 1.119970]
[6] [52/278] time: 0:38:55.775436, [d_loss: 0.273538, g_loss: 1.

[6] [153/278] time: 0:40:55.341628, [d_loss: 0.285479, g_loss: 0.979619]
[6] [154/278] time: 0:40:56.522409, [d_loss: 0.280641, g_loss: 1.139924]
[6] [155/278] time: 0:40:57.698592, [d_loss: 0.281525, g_loss: 1.047885]
[6] [156/278] time: 0:40:58.879140, [d_loss: 0.295822, g_loss: 1.289956]
[6] [157/278] time: 0:41:00.067481, [d_loss: 0.277756, g_loss: 1.153862]
[6] [158/278] time: 0:41:01.249083, [d_loss: 0.275173, g_loss: 1.099614]
[6] [159/278] time: 0:41:02.435996, [d_loss: 0.282417, g_loss: 1.220735]
[6] [160/278] time: 0:41:03.624149, [d_loss: 0.284689, g_loss: 1.171354]
[6] [161/278] time: 0:41:04.811641, [d_loss: 0.280156, g_loss: 1.161256]
[6] [162/278] time: 0:41:05.999652, [d_loss: 0.275630, g_loss: 1.225359]
[6] [163/278] time: 0:41:07.180058, [d_loss: 0.273222, g_loss: 1.240070]
[6] [164/278] time: 0:41:08.364212, [d_loss: 0.258918, g_loss: 1.251283]
[6] [165/278] time: 0:41:09.538556, [d_loss: 0.289160, g_loss: 1.123246]
[6] [166/278] time: 0:41:10.721154, [d_loss: 0.2824

[6] [266/278] time: 0:43:13.530471, [d_loss: 0.311735, g_loss: 1.035186]
[6] [267/278] time: 0:43:14.708572, [d_loss: 0.285236, g_loss: 1.296880]
[6] [268/278] time: 0:43:15.887476, [d_loss: 0.284032, g_loss: 1.299952]
[6] [269/278] time: 0:43:17.074006, [d_loss: 0.281214, g_loss: 1.221494]
[6] [270/278] time: 0:43:18.262041, [d_loss: 0.277471, g_loss: 1.195073]
[6] [271/278] time: 0:43:19.445476, [d_loss: 0.253659, g_loss: 1.580724]
[6] [272/278] time: 0:43:20.623166, [d_loss: 0.272223, g_loss: 1.326737]
[6] [273/278] time: 0:43:21.800128, [d_loss: 0.307588, g_loss: 1.179375]
[6] [274/278] time: 0:43:22.984839, [d_loss: 0.284773, g_loss: 1.225393]
[6] [275/278] time: 0:43:24.169810, [d_loss: 0.279433, g_loss: 1.074119]
[6] [276/278] time: 0:43:25.352009, [d_loss: 0.274545, g_loss: 1.270715]
[7] [0/278] time: 0:43:26.543449, [d_loss: 0.263837, g_loss: 1.321076]
[7] [1/278] time: 0:43:27.727792, [d_loss: 0.285179, g_loss: 1.143398]
[7] [2/278] time: 0:43:28.915699, [d_loss: 0.276524, g_

[7] [103/278] time: 0:45:28.545466, [d_loss: 0.273305, g_loss: 1.181087]
[7] [104/278] time: 0:45:29.734190, [d_loss: 0.283606, g_loss: 1.179213]
[7] [105/278] time: 0:45:30.919317, [d_loss: 0.292660, g_loss: 1.166398]
[7] [106/278] time: 0:45:32.117356, [d_loss: 0.306933, g_loss: 1.337963]
[7] [107/278] time: 0:45:33.295871, [d_loss: 0.277473, g_loss: 1.225347]
[7] [108/278] time: 0:45:34.471204, [d_loss: 0.280136, g_loss: 1.244304]
[7] [109/278] time: 0:45:35.661444, [d_loss: 0.272868, g_loss: 1.140787]
[7] [110/278] time: 0:45:36.844470, [d_loss: 0.268239, g_loss: 1.328590]
[7] [111/278] time: 0:45:38.040607, [d_loss: 0.279581, g_loss: 1.025662]
[7] [112/278] time: 0:45:39.249189, [d_loss: 0.276112, g_loss: 1.127380]
[7] [113/278] time: 0:45:40.448991, [d_loss: 0.262684, g_loss: 1.394539]
[7] [114/278] time: 0:45:41.636820, [d_loss: 0.272232, g_loss: 1.290241]
[7] [115/278] time: 0:45:42.855092, [d_loss: 0.282476, g_loss: 1.150179]
[7] [116/278] time: 0:45:44.050901, [d_loss: 0.2599

In [ ]:

if __name__ == '__main__':


    gan = colorizeSketch()
    
    gan.train(epochs=20, batch_size=4, sample_interval=200,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/209] time: 0:01:21.085297, [d_loss: 18.886578, g_loss: 68.600929]
[0] [1/209] time: 0:01:38.755886, [d_loss: 11.783155, g_loss: 205.253693]
[0] [2/209] time: 0:01:40.199998, [d_loss: 53.252922, g_loss: 70.752113]
[0] [3/209] time: 0:01:41.623814, [d_loss: 7.278514, g_loss: 23.199911]
[0] [4/209] time: 0:01:48.193272, [d_loss: 6.732075, g_loss: 6.095289]
[0] [5/209] time: 0:01:56.587872, [d_loss: 1.369521, g_loss: 5.633605]
[0] [6/209] time: 0:02:04.657860, [d_loss: 0.920789, g_loss: 5.474661]
[0] [7/209] time: 0:02:14.692328, [d_loss: 0.418258, g_loss: 4.970734]
[0] [8/209] time: 0:02:23.120946, [d_loss: 0.342002, g_loss: 4.327785]
[0] [9/209] time: 0:02:29.405413, [d_loss: 0.264540, g_loss: 4.522069]
[0] [10/209] time: 0:02:30.851710, [d_loss: 0.272102, g_loss: 4.050111]
[0] [11/209] time: 0:02:38.437381, [d_loss: 0.235956, g_loss: 3.908118]
[0] [12/209] time: 0:02:47.243615, [d_loss: 0.213401, g_loss: 4.440323]
[0] [13/209] time: 0:02:49.063250, [d_loss: 0.353477, g_loss: 4.10

[0] [114/209] time: 0:06:00.830268, [d_loss: 0.508848, g_loss: 2.465053]
[0] [115/209] time: 0:06:02.250825, [d_loss: 0.474058, g_loss: 2.312994]
[0] [116/209] time: 0:06:03.670773, [d_loss: 0.546817, g_loss: 1.960624]
[0] [117/209] time: 0:06:10.238073, [d_loss: 0.467566, g_loss: 2.491267]
[0] [118/209] time: 0:06:11.705637, [d_loss: 0.562123, g_loss: 2.303599]
[0] [119/209] time: 0:06:13.126452, [d_loss: 0.368552, g_loss: 2.275933]
[0] [120/209] time: 0:06:14.562546, [d_loss: 0.585240, g_loss: 3.518025]
[0] [121/209] time: 0:06:16.341269, [d_loss: 0.476437, g_loss: 2.370556]
[0] [122/209] time: 0:06:17.767949, [d_loss: 0.447453, g_loss: 2.743455]
[0] [123/209] time: 0:06:19.251092, [d_loss: 0.473527, g_loss: 2.623985]
[0] [124/209] time: 0:06:20.679948, [d_loss: 0.735497, g_loss: 3.139770]
[0] [125/209] time: 0:06:22.463526, [d_loss: 0.663300, g_loss: 2.227906]
[0] [126/209] time: 0:06:23.885984, [d_loss: 0.447344, g_loss: 2.317867]
[0] [127/209] time: 0:06:25.293464, [d_loss: 0.3899

[1] [19/209] time: 0:09:26.855073, [d_loss: 0.355777, g_loss: 2.117323]
[1] [20/209] time: 0:09:28.267803, [d_loss: 0.436981, g_loss: 1.848925]
[1] [21/209] time: 0:09:29.687850, [d_loss: 0.392980, g_loss: 1.977783]
[1] [22/209] time: 0:09:31.109620, [d_loss: 0.380830, g_loss: 2.212262]
[1] [23/209] time: 0:09:32.523427, [d_loss: 0.451544, g_loss: 1.793334]
[1] [24/209] time: 0:09:33.948579, [d_loss: 0.374148, g_loss: 2.005371]
[1] [25/209] time: 0:09:35.363554, [d_loss: 0.416829, g_loss: 1.824503]
[1] [26/209] time: 0:09:36.771618, [d_loss: 0.403041, g_loss: 1.737949]
[1] [27/209] time: 0:09:38.207048, [d_loss: 0.380063, g_loss: 1.856346]
[1] [28/209] time: 0:09:39.617392, [d_loss: 0.379182, g_loss: 2.043474]
[1] [29/209] time: 0:09:41.038570, [d_loss: 0.458095, g_loss: 1.636845]
[1] [30/209] time: 0:09:47.065836, [d_loss: 0.427167, g_loss: 1.706706]
[1] [31/209] time: 0:09:48.488080, [d_loss: 0.393992, g_loss: 1.774677]
[1] [32/209] time: 0:09:49.915118, [d_loss: 0.394096, g_loss: 1.

[1] [133/209] time: 0:12:34.191462, [d_loss: 0.376860, g_loss: 1.714004]
[1] [134/209] time: 0:12:35.590574, [d_loss: 0.384588, g_loss: 1.795846]
[1] [135/209] time: 0:12:37.000385, [d_loss: 0.421009, g_loss: 1.370049]
[1] [136/209] time: 0:12:43.137555, [d_loss: 0.390527, g_loss: 1.438720]
[1] [137/209] time: 0:12:44.545155, [d_loss: 0.320431, g_loss: 2.047113]
[1] [138/209] time: 0:12:45.954549, [d_loss: 0.453526, g_loss: 1.559992]
[1] [139/209] time: 0:12:47.352341, [d_loss: 0.361905, g_loss: 1.638467]
[1] [140/209] time: 0:12:48.759472, [d_loss: 0.346154, g_loss: 1.509731]
[1] [141/209] time: 0:12:50.167071, [d_loss: 0.401770, g_loss: 1.474561]
[1] [142/209] time: 0:12:51.574180, [d_loss: 0.362846, g_loss: 1.873267]
[1] [143/209] time: 0:12:53.014328, [d_loss: 0.405496, g_loss: 1.786773]
[1] [144/209] time: 0:12:54.420171, [d_loss: 0.396322, g_loss: 2.057055]
[1] [145/209] time: 0:12:55.818449, [d_loss: 0.397293, g_loss: 1.694465]
[1] [146/209] time: 0:12:57.218952, [d_loss: 0.4095

...

[4] [190/209] time: 0:29:31.509242, [d_loss: 0.288455, g_loss: 1.360035]
[4] [191/209] time: 0:29:32.922419, [d_loss: 0.276625, g_loss: 1.469370]
[4] [192/209] time: 0:29:34.327444, [d_loss: 0.298213, g_loss: 1.216209]
[4] [193/209] time: 0:29:35.731930, [d_loss: 0.291966, g_loss: 1.331125]
[4] [194/209] time: 0:29:37.131547, [d_loss: 0.297612, g_loss: 1.273605]
[4] [195/209] time: 0:29:38.532101, [d_loss: 0.294171, g_loss: 1.310925]
[4] [196/209] time: 0:29:39.943879, [d_loss: 0.262433, g_loss: 1.533670]
[4] [197/209] time: 0:29:41.352554, [d_loss: 0.268660, g_loss: 1.376254]
[4] [198/209] time: 0:29:42.766573, [d_loss: 0.285323, g_loss: 1.364017]
[4] [199/209] time: 0:29:44.176289, [d_loss: 0.316533, g_loss: 1.259393]
[4] [200/209] time: 0:29:45.580749, [d_loss: 0.285894, g_loss: 1.467109]
[4] [201/209] time: 0:29:47.011118, [d_loss: 0.327233, g_loss: 1.414963]
[4] [202/209] time: 0:29:48.415753, [d_loss: 0.305558, g_loss: 1.237424]
[4] [203/209] time: 0:29:49.821305, [d_loss: 0.3343

[5] [96/209] time: 0:32:12.333555, [d_loss: 0.275200, g_loss: 1.257277]
[5] [97/209] time: 0:32:13.744469, [d_loss: 0.273171, g_loss: 1.489271]
[5] [98/209] time: 0:32:15.146721, [d_loss: 0.291482, g_loss: 1.286018]
[5] [99/209] time: 0:32:16.562273, [d_loss: 0.296434, g_loss: 1.235969]
[5] [100/209] time: 0:32:17.975220, [d_loss: 0.280351, g_loss: 1.483755]
[5] [101/209] time: 0:32:19.380349, [d_loss: 0.294367, g_loss: 1.377096]
[5] [102/209] time: 0:32:20.805912, [d_loss: 0.291792, g_loss: 1.403110]
[5] [103/209] time: 0:32:22.224532, [d_loss: 0.292720, g_loss: 1.341172]
[5] [104/209] time: 0:32:23.635576, [d_loss: 0.279841, g_loss: 1.369106]
[5] [105/209] time: 0:32:25.052709, [d_loss: 0.302162, g_loss: 1.111306]
[5] [106/209] time: 0:32:26.462481, [d_loss: 0.284776, g_loss: 1.236171]
[5] [107/209] time: 0:32:27.872323, [d_loss: 0.295560, g_loss: 1.174544]
[5] [108/209] time: 0:32:29.286383, [d_loss: 0.294617, g_loss: 1.320004]
[5] [109/209] time: 0:32:30.699061, [d_loss: 0.255476, 

[6] [1/209] time: 0:35:01.694665, [d_loss: 0.289640, g_loss: 1.191648]
[6] [2/209] time: 0:35:03.113295, [d_loss: 0.295735, g_loss: 1.160758]
[6] [3/209] time: 0:35:04.519176, [d_loss: 0.272161, g_loss: 1.318554]
[6] [4/209] time: 0:35:05.925807, [d_loss: 0.285227, g_loss: 1.250510]
[6] [5/209] time: 0:35:07.334494, [d_loss: 0.275915, g_loss: 1.367628]
[6] [6/209] time: 0:35:08.743671, [d_loss: 0.279685, g_loss: 1.349966]
[6] [7/209] time: 0:35:10.161166, [d_loss: 0.273867, g_loss: 1.214367]
[6] [8/209] time: 0:35:11.568265, [d_loss: 0.286630, g_loss: 1.181602]
[6] [9/209] time: 0:35:12.983925, [d_loss: 0.267596, g_loss: 1.273744]
[6] [10/209] time: 0:35:14.387679, [d_loss: 0.293759, g_loss: 1.146565]
[6] [11/209] time: 0:35:15.807488, [d_loss: 0.290938, g_loss: 1.136621]
[6] [12/209] time: 0:35:17.213664, [d_loss: 0.273766, g_loss: 1.207870]
[6] [13/209] time: 0:35:18.632660, [d_loss: 0.298924, g_loss: 1.290197]
[6] [14/209] time: 0:35:20.057528, [d_loss: 0.279173, g_loss: 1.352849]
[

[6] [115/209] time: 0:37:42.268697, [d_loss: 0.286830, g_loss: 1.157650]
[6] [116/209] time: 0:37:43.681592, [d_loss: 0.289812, g_loss: 1.049238]
[6] [117/209] time: 0:37:45.099950, [d_loss: 0.289225, g_loss: 1.254311]
[6] [118/209] time: 0:37:46.499646, [d_loss: 0.280858, g_loss: 1.240945]
[6] [119/209] time: 0:37:47.904864, [d_loss: 0.279484, g_loss: 1.166844]
[6] [120/209] time: 0:37:49.307544, [d_loss: 0.281151, g_loss: 1.205605]
[6] [121/209] time: 0:37:50.711200, [d_loss: 0.284433, g_loss: 1.279265]
[6] [122/209] time: 0:37:52.115442, [d_loss: 0.272854, g_loss: 1.321452]
[6] [123/209] time: 0:37:53.521990, [d_loss: 0.246359, g_loss: 1.308727]
[6] [124/209] time: 0:37:54.928237, [d_loss: 0.282948, g_loss: 1.165955]
[6] [125/209] time: 0:37:56.336151, [d_loss: 0.286593, g_loss: 1.187658]
[6] [126/209] time: 0:37:57.801855, [d_loss: 0.287237, g_loss: 1.071887]
[6] [127/209] time: 0:37:59.227487, [d_loss: 0.280939, g_loss: 1.136717]
[6] [128/209] time: 0:38:00.626864, [d_loss: 0.2524

[7] [20/209] time: 0:40:26.984203, [d_loss: 0.271340, g_loss: 1.127045]
[7] [21/209] time: 0:40:28.411061, [d_loss: 0.276960, g_loss: 1.258527]
[7] [22/209] time: 0:40:29.846941, [d_loss: 0.259908, g_loss: 1.313130]
[7] [23/209] time: 0:40:31.280893, [d_loss: 0.266407, g_loss: 1.238662]
[7] [24/209] time: 0:40:32.689811, [d_loss: 0.261066, g_loss: 1.243888]
[7] [25/209] time: 0:40:34.113734, [d_loss: 0.287681, g_loss: 1.196193]
[7] [26/209] time: 0:40:35.521017, [d_loss: 0.284595, g_loss: 1.161873]
[7] [27/209] time: 0:40:36.959324, [d_loss: 0.279120, g_loss: 1.199145]
[7] [28/209] time: 0:40:38.396198, [d_loss: 0.268785, g_loss: 1.239415]
[7] [29/209] time: 0:40:39.816070, [d_loss: 0.286156, g_loss: 1.134659]
[7] [30/209] time: 0:40:41.284350, [d_loss: 0.278154, g_loss: 1.139981]
[7] [31/209] time: 0:40:42.710937, [d_loss: 0.280728, g_loss: 1.184717]
[7] [32/209] time: 0:40:44.121817, [d_loss: 0.265082, g_loss: 1.255901]
[7] [33/209] time: 0:40:45.524103, [d_loss: 0.271765, g_loss: 1.